In [1]:
import os
import requests
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm

import sys
sys.path.append(r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download')
from utils import extract_velocity_links_from_toc_html, download_velocity_files

In [61]:
extract_velocity_links_from_toc_html(
    raw_toc_fpath=r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download/raw_table_of_contents_V1-velocity.txt', 
    output_fpath=r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download/download_urls_V1-velocity.csv',
    fname_pattern=r'velocity[0-9]+-[0-9]+\.feather'
)

extract_velocity_links_from_toc_html(
    raw_toc_fpath=r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download/raw_table_of_contents_V2-velocity.txt', 
    output_fpath=r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download/download_urls_V2-velocity.csv',
    fname_pattern=r'velocity[0-9]+-[0-9]+\.zip'
)

[INFO] Found 300 matches.
[INFO] Found 300 matches.


In [ ]:
download_velocity_files(
    download_urls_fpath=r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download/download_urls_V1-velocity.csv',
    save_dir=r'/cluster/work/math/camlab-data/Wave_HemewS-3D/version1/velocity',
    max_downloads=None
)
download_velocity_files(
    download_urls_fpath=r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download/download_urls_V2-velocity.csv',
    save_dir=r'/cluster/work/math/camlab-data/Wave_HemewS-3D/version2/velocity',
    max_downloads=None
)

[INFO] Found 300 files to download. 61 files already downloaded. Downloading the remaining 239 files.



## Downloading single faulty file

In [4]:
download_urls_fpath=r'/cluster/home/lcarretero/workspace/rds/HEMEW3D/GenCFD-prep/data-download/download_urls_V1-velocity.csv'
df = pd.read_csv(download_urls_fpath)
df.head()

,link_element,persistent_id,fname,download_link
0,"<a href=""/file.xhtml?persistentId=doi:10.57745...",7XMGCR,velocity0-99.feather,https://entrepot.recherche.data.gouv.fr/api/ac...
1,"<a href=""/file.xhtml?persistentId=doi:10.57745...",JZBA9R,velocity100-199.feather,https://entrepot.recherche.data.gouv.fr/api/ac...
2,"<a href=""/file.xhtml?persistentId=doi:10.57745...",QGQNSJ,velocity1000-1099.feather,https://entrepot.recherche.data.gouv.fr/api/ac...
3,"<a href=""/file.xhtml?persistentId=doi:10.57745...",HWDFKC,velocity10000-10099.feather,https://entrepot.recherche.data.gouv.fr/api/ac...
4,"<a href=""/file.xhtml?persistentId=doi:10.57745...",SXTQKG,velocity10100-10199.feather,https://entrepot.recherche.data.gouv.fr/api/ac...


In [7]:
faulty_file_name = 'velocity8500-8599.feather'
url = df[df.fname==faulty_file_name].download_link.values[0]
print(url)

https://entrepot.recherche.data.gouv.fr/api/access/datafile/:persistentId?persistentId=doi:10.57745/6R65AA


In [8]:
save_dir = r'/cluster/work/math/camlab-data/Wave_HemewS-3D/version1/velocity'
fname = faulty_file_name

os.remove(os.path.join(save_dir, fname))

response = requests.get(url, stream=True)
response.raise_for_status()
with open(os.path.join(save_dir, fname), 'wb') as f:
    try:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    except Exception as e:
        print(f'[ERROR] Error downloading file {fname}: {e}')
        os.remove(os.path.join(save_dir, fname))